In [ ]:
import DeepFMKit.core as dfm
from DeepFMKit import physics, experiments
from DeepFMKit.workers import calculate_wdfmi_vs_dfmi_bias
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
from matplotlib.colors import LinearSegmentedColormap
from scipy import constants as sc
from tqdm import tqdm
import multiprocessing
import os
import copy # Import the copy module

In [ ]:
def validate_wdfmi_bias_correction(
    m_main=15.5,
    m_witness=0.05,
    ndata=20,
    distortion_range=np.linspace(0, 0.05, 11),
    n_phase_trials=50,
    n_cores=None
):
    """
    Generates a plot to validate the bias correction capability of W-DFMI.

    This function orchestrates a parallel Monte Carlo simulation. For each
    level of signal distortion, it calls the `run_monte_carlo` runner, which
    repeatedly executes the `calculate_wdfmi_vs_dfmi_bias` worker with
    randomized distortion phases.

    The results are then aggregated to plot the mean and standard deviation
    of the bias for each fitter as a function of distortion amplitude.
    """
    if n_cores is None:
        n_cores = os.cpu_count()

    print("=" * 60)
    print("Validating W-DFMI Bias Correction vs. Modulation Non-Linearity")
    print(f"Parameters: m_main={m_main}, m_witness={m_witness}, ndata={ndata}")
    print("=" * 60)

    # Lists to store the final aggregated statistics for plotting
    dfmi_bias_mean, dfmi_bias_std = [], []
    wdfmi_bias_mean, wdfmi_bias_std = [], []
    
    # --- Base configurations that are the same for all jobs ---
    base_laser_config = physics.LaserConfig()
    main_ifo_config = physics.InterferometerConfig()
    opd_main = main_ifo_config.meas_arml - main_ifo_config.ref_arml
    base_laser_config.df = (m_main * sc.c) / (2 * np.pi * opd_main)
    
    # --- Outer Loop: Iterate over distortion amplitude ---
    for eps in tqdm(distortion_range, desc="Scanning Distortion Level"):
        
        # --- 1. Set up the parameters for the Monte Carlo run ---
        static_params = {
            'main_ifo_config': main_ifo_config,
            'm_main': m_main,
            'm_witness': m_witness,
            'ndata': ndata,
        }

        # A function that generates the dynamic (random) part of the params
        def dynamic_params_generator(trial_idx):
            # Create a shallow copy of the laser config for this specific trial.
            # This prevents all trials from sharing the same mutable object.
            trial_laser_config = copy.copy(base_laser_config)
            
            # Now, modify the *copy* with the unique parameters for this trial.
            trial_laser_config.df_2nd_harmonic_frac = eps
            trial_laser_config.df_2nd_harmonic_phase = np.random.uniform(0, 2 * np.pi)
            
            return {'laser_config': trial_laser_config}

        # --- 2. Execute the Monte Carlo run for this `eps` ---
        results_for_eps = experiments.run_monte_carlo(
            worker_func=calculate_wdfmi_vs_dfmi_bias,
            n_trials=n_phase_trials,
            static_params=static_params,
            dynamic_params_generator=dynamic_params_generator,
            verbose=False,
            n_cores=n_cores
        ) # Shape of results_for_eps is (n_trials, 2)

        # --- 3. Calculate and store statistics for this `eps` level ---
        # Column 0 is W-DFMI bias, Column 1 is standard NLS bias
        wdfmi_bias_mean.append(np.nanmean(results_for_eps[:, 0]))
        wdfmi_bias_std.append(np.nanstd(results_for_eps[:, 0]))
        
        dfmi_bias_mean.append(np.nanmean(results_for_eps[:, 1]))
        dfmi_bias_std.append(np.nanstd(results_for_eps[:, 1]))

    # --- 4. Plotting ---
    fig, ax = plt.subplots(figsize=(12, 7))

    ax.errorbar(distortion_range * 100, dfmi_bias_mean, yerr=dfmi_bias_std,
                fmt='o-', capsize=5, color='tab:red', label='Standard NLS Fitter Bias')

    ax.errorbar(distortion_range * 100, wdfmi_bias_mean, yerr=wdfmi_bias_std,
                fmt='s-', capsize=5, color='tab:green', label='W-DFMI Fitter Bias (Corrected)')

    ax.axhline(0, color='k', linestyle='--', linewidth=1, alpha=0.7)
    ax.set_xlabel('2nd Harmonic Distortion Amplitude (%)', fontsize=14)
    ax.set_ylabel(r"Bias in Modulation Depth, $\delta m = \hat{m} - m_{\rm true}$ (rad)", fontsize=14)
    ax.set_title('W-DFMI Correction of Systematic Bias from Modulation Non-Linearity', fontsize=16)
    ax.grid(True, which='both', linestyle=':')
    ax.legend(fontsize=12)
    plt.tight_layout()
    plt.show()

    return ax

In [ ]:
ax = validate_wdfmi_bias_correction(m_main=20.3, m_witness=0.1, ndata=15, distortion_range=np.linspace(0.0001, 0.1, 5), n_phase_trials=100)
plt.show()

In [ ]:
import DeepFMKit.core as dfm
from DeepFMKit import physics, runner
# Import the helper functions from the workers module
from DeepFMKit.workers import (
    setup_distortion_trial, 
    get_random_distortion_phase, 
    extract_bias
)
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap
from matplotlib.gridspec import GridSpec
from scipy import constants as sc

def generate_and_plot_bias_landscape():
    """
    Defines, runs, and plots the W-DFMI bias landscape experiment.
    """
    print("=" * 60)
    print("Generating Full Bias Landscape Data using Experiment Runner")
    print("=" * 60)
    
    # --- 1. Declaratively Define the Experiment ---
    exp = runner.Experiment(description="Systematic Bias from M|odulation Non-Linearity")

    # Define the 2D parameter grid
    exp.add_axis('m_main', np.linspace(2, 25, 16))
    exp.add_axis('distortion_amp', np.linspace(0, 0.2, 16))

    # Define static parameters
    exp.set_static({'m_witness': 0.05})

    # Define the Monte Carlo simulation
    exp.n_trials = 10
    exp.add_stochastic_variable('distortion_phase', get_random_distortion_phase)

    # Define the physics setup for a single trial
    exp.set_trial_setup(setup_distortion_trial)

    # Define the analyses to run
    exp.add_analysis(
        name='wdfmi_bias',
        fitter_method='wdfmi_ortho',
        result_extractor_func=extract_bias
    )
    exp.add_analysis(
        name='nls_bias',
        fitter_method='nls',
        fitter_kwargs={'ndata': 35, 'parallel': False}, # Max ndata for m=25
        result_extractor_func=extract_bias
    )

    # --- 2. Run the Experiment ---
    results = exp.run()

    # --- 3. Plot the Results ---
    if results is None:
        print("Experiment failed to produce results.")
        return

    m_range = results['axes']['m_main']
    distortion_range = results['axes']['distortion_amp']
    
    fig = plt.figure(figsize=(18, 14))
    gs = GridSpec(2, 3, width_ratios=[1, 1, 0.05], wspace=0.3, hspace=0.4)
    ax1, ax2 = fig.add_subplot(gs[0, 0]), fig.add_subplot(gs[0, 1])
    ax3, ax4 = fig.add_subplot(gs[1, 0]), fig.add_subplot(gs[1, 1])
    cax = fig.add_subplot(gs[:, 2])

    colors = ["#FFFFFF", "#D6EAF8", "#5499C7", "#154360"]
    custom_cmap = LinearSegmentedColormap.from_list("custom_cmap", colors)
    vmax = np.pi

    def plot_panel(ax, data, title):
        # --- THE FIX IS HERE ---
        # The data grid must be transposed (.T) to align its dimensions
        # with the X (m_range) and Y (distortion_range) coordinate arrays.
        im = ax.pcolormesh(m_range, distortion_range * 100, np.abs(data.T), 
                           cmap=custom_cmap, vmin=0, vmax=vmax, shading='auto')
        ax.set_title(title, fontsize=16, pad=10)
        ax.set_xlabel(r'Modulation Depth, $m$', fontsize=14)
        ax.set_ylabel(r'2nd Harmonic Distortion, $\epsilon$ (%)', fontsize=14)
        return im

    plot_panel(ax1, results['nls_bias']['mean'], '(a) Standard NLS: Mean Bias')
    plot_panel(ax2, results['wdfmi_bias']['mean'], '(b) W-DFMI: Mean Bias (Corrected)')
    plot_panel(ax3, results['nls_bias']['worst'], '(c) Standard NLS: Worst-Case Bias')
    im = plot_panel(ax4, results['wdfmi_bias']['worst'], '(d) W-DFMI: Worst-Case Bias (Corrected)')

    fig.colorbar(im, cax=cax, orientation='vertical', label=r'Absolute Bias, $|\delta m|$ (rad)')
    plt.show()

# --- Entry point for execution ---
if __name__ == "__main__":
    generate_and_plot_bias_landscape()